## 라이브러리 생성

In [ ]:
# !pip install eli5==0.13.0
%pip install eli5==0.13.0
%pip install haversine
%pip install optuna xgboost


# 한글 폰트 사용을 위한 라이브러리입니다.
# !apt-get install -y fonts-nanum
!apt-get install -y fonts-nanum

!pip install lightgbm==4.0.0

In [ ]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GroupKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

# 추가

from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb

# 데이터셋 로드 및 정제
import pandas as pd
import joblib
import os


# import eli5
# from eli5.sklearn import PermutationImportance

## 데이터 로드

In [ ]:
total_data = '/data/ephemeral/home/house-price/data/merge_subway_fixed.csv'
df = pd.read_csv(total_data)

In [ ]:
print('Total data shape : ', df.shape)

In [ ]:
df.isnull().sum()

In [ ]:
train_df = df[df['is_test'] == 0]
test_df  = df[df['is_test'] == 1]

# 크기 확인
print("Train 데이터 행 개수:", train_df.shape[0])
print("Test 데이터 행 개수:", test_df.shape[0])

## 데이터 전처리 

#### 좌표 결측치 처리

In [ ]:
df.isnull().sum()

In [ ]:
# ---------------------------
# 1. Train/Test 분리 (이 부분은 동일합니다)
# ---------------------------
train_df = df[df["is_test"] == 0].copy()
test_df  = df[df["is_test"] == 1].copy()

print("Before:", train_df.shape, test_df.shape)

# ---------------------------
# 2. 훈련 세트(train_df)에서 결측치 처리 규칙 찾기
# ---------------------------

# (1) 결측치가 많은 행 삭제 (훈련 세트에만 적용)
train_df.dropna(subset=["좌표X", "좌표Y", "가까운역이름", "거리"], how="all", inplace=True)

# (2) 결측치 대체를 위한 통계값 계산 (✨훈련 데이터'로만' 계산)
sigungu_coord_means = train_df.groupby("시군구")[["좌표X", "좌표Y"]].transform('mean')
sigungu_dist_medians = train_df.groupby("시군구")["거리"].transform('median')
global_dist_median = train_df["거리"].median() # 전체 거리 중앙값

# (3) 계산된 통계값으로 훈련 데이터(train_df)의 결측치 채우기
train_df["좌표X"].fillna(sigungu_coord_means["좌표X"], inplace=True)
train_df["좌표Y"].fillna(sigungu_coord_means["좌표Y"], inplace=True)
train_df["가까운역이름"].fillna("Unknown", inplace=True)
train_df["거리"].fillna(sigungu_dist_medians, inplace=True)
train_df["거리"].fillna(global_dist_median, inplace=True) # 시군구 중앙값도 없는 경우 대비

# ---------------------------
# 3. 테스트 세트(test_df)에 '훈련 세트의 규칙' 적용하기
# ---------------------------

# (1) 테스트 데이터의 통계값 계산 (✨'훈련 데이터'의 규칙을 적용하기 위해)
test_sigungu_coord_means = train_df.groupby('시군구')[['좌표X', '좌표Y']].mean() # 시군구별 평균 좌표
test_sigungu_dist_medians = train_df.groupby('시군구')['거리'].median() # 시군구별 거리 중앙값

# (2) 테스트 데이터의 시군구별 결측치를 훈련 데이터의 평균/중앙값으로 채우기
test_df['좌표X'] = test_df.set_index('시군구')['좌표X'].fillna(test_sigungu_coord_means['좌표X']).values
test_df['좌표Y'] = test_df.set_index('시군구')['좌표Y'].fillna(test_sigungu_coord_means['좌표Y']).values
test_df['거리'] = test_df.set_index('시군구')['거리'].fillna(test_sigungu_dist_medians).values

# (3) 그래도 남은 결측치는 훈련 데이터의 전체 중앙값(global_dist_median)으로 채우기
test_df["좌표X"].fillna(train_df['좌표X'].mean(), inplace=True) # 훈련 데이터에 없는 시군구 대비
test_df["좌표Y"].fillna(train_df['좌표Y'].mean(), inplace=True) # 훈련 데이터에 없는 시군구 대비
test_df["가까운역이름"].fillna("Unknown", inplace=True)
test_df["거리"].fillna(global_dist_median, inplace=True)

print("After:", train_df.shape, test_df.shape)

# ---------------------------
# 4. 다시 합치기
# ---------------------------
df = pd.concat([train_df, test_df], ignore_index=True)

print("최종 데이터셋 크기:", df.shape)
print("남은 결측치 수:\n", df[["좌표X", "좌표Y", "가까운역이름", "거리"]].isna().sum())

In [ ]:
df.isnull().sum()

In [ ]:
fig = plt.figure(figsize=(13, 2))
missing = df.isnull().sum() / df.shape[0]
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar(color='orange')
plt.title('변수별 결측치 비율')
plt.show()

## 이상치

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 수치형 컬럼만 선택
num_cols = df.select_dtypes(include=['float64', 'int64']).columns

# 긴 형태로 변환 (컬럼명 -> variable, 값 -> value)
df_melt = df[num_cols].melt(var_name="Feature", value_name="Value")

# 박스플롯
plt.figure(figsize=(15, 6))
sns.boxplot(data=df_melt, x="Feature", y="Value")
plt.xticks(rotation=90)  # 가독성을 위해 x축 라벨 회전
plt.title("Boxplots of All Numeric Features")
plt.show()


## 파생변수 생성

#### 시군구 분리

In [ ]:
df['is_test'].value_counts()     # 또한, train data만 제거되었습니다.

In [ ]:
df['시군구'].map(lambda x : x.split()[2])

In [ ]:
df['계약년월'].astype('str').map(lambda x : x[:4])

In [ ]:
# 시군구, 년월 등 분할할 수 있는 변수들은 세부사항 고려를 용이하게 하기 위해 모두 분할해 주겠습니다.
df['구'] = df['시군구'].map(lambda x : x.split()[1])
df['동'] = df['시군구'].map(lambda x : x.split()[2])
del df['시군구']

df['계약년'] = df['계약년월'].astype('str').map(lambda x : x[:4])
df['계약월'] = df['계약년월'].astype('str').map(lambda x : x[4:])
del df['계약년월']

In [ ]:
df.columns

In [ ]:
# 연속형 변수: ['전용면적', '계약일', '층', '건축년도', 'k-전체동수', 'k-전체세대수', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', '건축면적', '주차대수', '좌표X', '좌표Y', 'target', '강남여부', '신축여부']
# 범주형 변수: ['번지', '본번', '부번', '아파트명', '도로명', 'k-단지분류(아파트,주상복합등등)', 'k-전화번호', 'k-팩스번호', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '단지신청일', '구', '동', '계약년', '계약월']
del_features = ['부번', '계약일', '계약월']
df_ft = df.drop(del_features, axis=1)


In [ ]:
df_ft.columns

#### 전용면적

In [ ]:
# ---------------------------
# 전용면적 관련 파생변수
# ---------------------------

# 1) 로그 변환 (큰 값의 분산 완화)
df_ft['log_전용면적'] = np.log1p(df_ft['전용면적'])

# 2) 전용면적 구간화 (소형/중형/대형/초대형)
bins = [0, 60, 85, 135, np.inf]  # m² 기준
labels = ['소형', '중형', '대형', '초대형']
df_ft['전용면적_구간'] = pd.cut(df_ft['전용면적'], bins=bins, labels=labels)

# # 3) 평당가격 (㎡를 평으로 변환 후 계산)
# df_ft['평당가격'] = df_ft['target'] / (df_ft['전용면적'] / 3.3)


In [ ]:
df_ft['단지명'] = df_ft['동'] + "_" + df_ft['아파트명']

#### 거리 변수

In [ ]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
from tqdm import tqdm

# 두 좌표 사이 거리 계산 (m 단위)
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371000  # 지구 반경 (m)
    return c * r

closest_names = []
closest_distances = []
station_counts = []

# tqdm으로 진행률 표시
for idx, row in tqdm(df_ft.iterrows(), total=len(df_ft), desc="Processing"):
    if pd.isna(row['좌표X']) or pd.isna(row['좌표Y']):
        closest_names.append(None)
        closest_distances.append(None)
        station_counts.append(0)
        continue
    
    apt_lon = float(row['좌표X'])
    apt_lat = float(row['좌표Y'])
    
    # 지하철역들과 거리 계산
    distances = df.apply(
        lambda r: haversine(apt_lon, apt_lat, float(r['경도']), float(r['위도'])), axis=1
    )
    
    # 가장 가까운 역
    min_idx = distances.idxmin()
    min_dist = distances[min_idx]
    min_station = df.loc[min_idx, '역사명']
    
    # 1km 이내 역 개수
    count_within_1km = (distances <= 1000).sum()
    
    closest_names.append(min_station)
    closest_distances.append(int(round(min_dist / 10) * 10))  # 10m 단위 반올림
    station_counts.append(count_within_1km)

# 새로운 열 추가
concat_merge = df_ft.copy()
concat_merge['가까운역이름'] = closest_names
concat_merge['거리'] = closest_distances
concat_merge['1km내역개수'] = station_counts


## 모델링

## 범주형 인코딩 및 lightGBM 모델 사용

#### 범주형 인코딩

In [ ]:
import numpy as np
import joblib
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

# ---------------------------
# 1. Train/Test 분리
# ---------------------------
train_df = df_ft[df_ft['is_test'] == 0].drop(columns=['is_test'])
test_df  = df_ft[df_ft['is_test'] == 1].drop(columns=['is_test'])

X_train = train_df.drop(columns=['target'])
y_train = np.log1p(train_df['target'])

# 테스트 데이터에서도 동일하게 제거
X_test  = test_df.drop(columns=['target'])
# X_train 안에서 object 타입 컬럼만 확인
object_cols = X_train.select_dtypes(include="object").columns.tolist()
print("Object 타입 컬럼:", object_cols)

# 개수 확인
print("총 개수:", len(object_cols))

# ---------------------------
# 2. object → category 변환
# ---------------------------
for col in X_train.columns:
    if X_train[col].dtype == "object":
        X_train[col] = X_train[col].astype("category")
        X_test[col]  = X_test[col].astype("category")

#### 최적화 여부

In [ ]:
# import optuna

# def objective(trial):
#     param = {
#         "objective": "regression",
#         "metric": "rmse",
#         "n_estimators": 2000,
#         "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
#         "num_leaves": trial.suggest_int("num_leaves", 31, 255),
#         "max_depth": trial.suggest_int("max_depth", 3, 12),
#         "subsample": trial.suggest_float("subsample", 0.6, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
#         "random_state": 42
#     }
#     gbm = lgb.LGBMRegressor(**param)
#     gbm.fit(X_train, y_train)
#     y_pred = gbm.predict(X_train)
#     return mean_squared_error(y_train, y_pred, squared=False)

# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=30)
# print("Best params:", study.best_params)


#### 모델링

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import joblib
import numpy as np
import pandas as pd

# ---------------------------
# 3. TimeSeriesSplit 학습 + fold별 RMSE 기록
# ---------------------------
kf = TimeSeriesSplit(n_splits=5)
fold_save_files = []
cv_results = []  # fold별 성능 저장용

for fold_idx, (train_idx, valid_idx) in enumerate(kf.split(X_train)):
    print(f"\n======== {fold_idx}번째 fold 학습 시작 ========")

    X_train_fold, Y_train_fold = X_train.iloc[train_idx].copy(), y_train.iloc[train_idx].copy()
    X_valid_fold, Y_valid_fold = X_train.iloc[valid_idx].copy(), y_train.iloc[valid_idx].copy()

    # ---------------------------
    # Fold 내 luxury_apt 생성 (누수 방지)
    # ---------------------------
    temp_train = X_train_fold.copy()
    temp_train['target'] = np.expm1(Y_train_fold)

    if '2023' in temp_train['계약년'].unique():
        luxury_condo = temp_train[temp_train['계약년'] == '2023'].groupby('단지명')['target'].mean()
    else:
        luxury_condo = temp_train.groupby('단지명')['target'].mean()

    luxury_list = luxury_condo[luxury_condo >= 200000].index
    X_train_fold['luxury_apt'] = X_train_fold['단지명'].isin(luxury_list).astype(int)
    X_valid_fold['luxury_apt'] = X_valid_fold['단지명'].isin(luxury_list).astype(int)

    # ---------------------------
    # 모델 정의 및 학습
    # ---------------------------
    gbm = lgb.LGBMRegressor(
        learning_rate=0.05,
        num_leaves=20,
        max_depth=6,
        min_child_samples=30,       # 추가: 과적합 제어
        lambda_l1=1.0,              # 추가: L1 규제
        lambda_l2=1.0,              # 추가: L2 규제
        subsample=0.7495,
        colsample_bytree=0.7565,
        n_estimators=10000,
        random_state=42,
        n_jobs=-1
    )

    gbm.fit(
        X_train_fold, Y_train_fold,
        eval_set=[(X_train_fold, Y_train_fold), (X_valid_fold, Y_valid_fold)],
        eval_metric="rmse",
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=100)
        ]
    )

    # ---------------------------
    # 예측 및 RMSE 계산
    # ---------------------------
    y_train_pred_log = gbm.predict(X_train_fold, num_iteration=gbm.best_iteration_)
    y_valid_pred_log = gbm.predict(X_valid_fold, num_iteration=gbm.best_iteration_)

    # exp로 되돌림
    y_train_pred_real = np.expm1(y_train_pred_log)
    y_valid_pred_real = np.expm1(y_valid_pred_log)
    y_train_true_real = np.expm1(Y_train_fold)
    y_valid_true_real = np.expm1(Y_valid_fold)

    # RMSE 계산
    rmse_train_log = np.sqrt(mean_squared_error(Y_train_fold, y_train_pred_log))
    rmse_train_real = np.sqrt(mean_squared_error(y_train_true_real, y_train_pred_real))
    rmse_valid_log = np.sqrt(mean_squared_error(Y_valid_fold, y_valid_pred_log))
    rmse_valid_real = np.sqrt(mean_squared_error(y_valid_true_real, y_valid_pred_real))

    cv_results.append({
        "fold": fold_idx,
        "train_log_RMSE": rmse_train_log,
        "train_real_RMSE": rmse_train_real,
        "valid_log_RMSE": rmse_valid_log,
        "valid_real_RMSE": rmse_valid_real
    })

    print(f"✅ Fold {fold_idx}")
    print(f"   Train 로그 RMSE: {rmse_train_log:.4f}, Train 원래 RMSE: {rmse_train_real:.2f}")
    print(f"   Valid 로그 RMSE: {rmse_valid_log:.4f}, Valid 원래 RMSE: {rmse_valid_real:.2f}")

    # 모델 저장
    file_name = f"timeseries_fold{fold_idx}_gbm.pkl"
    joblib.dump(gbm, file_name)
    fold_save_files.append(file_name)

# ---------------------------
# 4. 최종 요약
# ---------------------------
cv_df = pd.DataFrame(cv_results)
print("\n======== 교차검증 결과 요약 ========")
print(cv_df)

print("\n평균 Valid 로그 RMSE:", cv_df["valid_log_RMSE"].mean())
print("평균 Valid 원래 RMSE:", cv_df["valid_real_RMSE"].mean())
print("최적(가장 낮은) Valid 로그 RMSE:", cv_df["valid_log_RMSE"].min())
print("최적(가장 낮은) Valid 원래 RMSE:", cv_df["valid_real_RMSE"].min())


In [ ]:
# ---------------------------
# 5. Feature Importance 분석
# ---------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# --- 수정된 부분 시작 ---

# ✅ 1. 'luxury_apt'를 포함한 최종 특성 개수로 importances 배열을 초기화합니다.
num_final_features = X_train.shape[1] + 1
importances = np.zeros(num_final_features)

for f in fold_save_files:
    model = joblib.load(f)
    # 이제 양쪽의 shape가 (24,) (24,)로 동일해져서 오류가 발생하지 않습니다.
    importances += model.feature_importances_

importances /= len(fold_save_files)

# ✅ 2. DataFrame을 만들 때 사용할 최종 특성 이름 리스트를 만듭니다.
# 기존 X_train의 컬럼에 'luxury_apt'를 추가합니다.
final_feature_names = X_train.columns.tolist() + ['luxury_apt']

importance_df = pd.DataFrame({
    "feature": final_feature_names, # 수정된 특성 이름 리스트 사용
    "importance": importances
}).sort_values(by="importance", ascending=False)

# --- 수정된 부분 끝 ---

plt.figure(figsize=(12,6))
sns.barplot(data=importance_df.head(20), x="importance", y="feature")
plt.title("Top 20 Feature Importance")
plt.show()

## 출력파일 생성

In [ ]:
# preds = []
# for f in fold_save_files:
#     model = joblib.load(f)
#     preds.append(model.predict(X_test, num_iteration=model.best_iteration_))

# y_test_pred = np.expm1(np.mean(preds, axis=0))

# submission = pd.read_csv("/data/ephemeral/home/house-price/data/sample_submission.csv")
# submission["target"] = np.round(y_test_pred).astype(int)
# submission.to_csv("submission_lgb_log.csv", index=False)

# print("✅ 제출 파일 저장 완료: submission_lgb_log.csv")
